# dev-tooling.ipynb

Develop the API code to support tooling

In [1]:
import os, sys
from typing import Optional, List, Dict
sys.path.append(os.path.join(".."))
import hjson
import docstring_parser
import inspect
from llms_wrapper.llms import LLMS
from llms_wrapper.config import update_llm_config

/home/johann/software/miniconda3/envs/llms_wrapper/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        # dict(llm="openai/o1"),        # restricted
        # dict(llm="openai/o1-mini"),   # restricted
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
_ = update_llm_config(config)

In [3]:
llms = LLMS(config)

In [4]:
llms.list_aliases()

['openai/gpt-4o',
 'openai/gpt-4o-mini',
 'gemini/gemini-2.0-flash-exp',
 'gemini/gemini-1.5-flash',
 'gemini/gemini-1.5-pro',
 'anthropic/claude-3-5-sonnet-20240620',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-beta',
 'groq/llama3-70b-8192',
 'groq/llama-3.3-70b-versatile',
 'deepseek/deepseek-chat']

In [10]:
def query_names(where_clause: str) -> List[str]: 
    """
    Query the customer database and return a list of matching names. 

    This function queries the customer database using the conditions in the where clause and returns
    a list of matching customers. The where clause may use the DB fields "city", "company_name",
    "country" and "since_date" to limit the returned customer list. The where clause can also 
    be followed by a limit clause to limit the number of returned names. 

    :param string where_clause: the string containing the where and optionally limit clauses in SQL query format
    :return: a list of matching customer names
    :rtype: list of strings
    """
    return ["Monica Schmidt", "Harald Mueller"]


In [11]:
tools=llms.make_tooling(query_names)
tools

[{'type': 'function',
  'function': {'name': 'query_names',
   'description': 'Query the customer database and return a list of matching names. \n\nThis function queries the customer database using the conditions in the where clause and returns\na list of matching customers. The where clause may use the DB fields "city", "company_name",\n"country" and "since_date" to limit the returned customer list. The where clause can also \nbe followed by a limit clause to limit the number of returned names.',
   'parameters': {'type': 'object',
    'properties': {'where_clause': {'type': 'string',
      'description': 'the string containing the where and optionally limit clauses in SQL query format'}},
    'required': ['where_clause']}}}]

In [12]:
llms.supports_function_calling("openai/gpt-4o")

True

In [13]:
msgs = llms.make_messages("Give me the names of customers in New York which have been customers since 2023 or longer")

In [14]:
ret = llms.query("openai/gpt-4o", messages=msgs, tools=tools)
ret["answer"], ret["error"]

(None, '')

In [16]:
ret["tool_calls"]

[ChatCompletionMessageToolCall(function=Function(arguments='{"where_clause":"WHERE city = \'New York\' AND since_date <= \'2023-01-01\'"}', name='query_names'), id='call_AR1LI5hcO09BrvYxyS81zvOT', type='function')]

In [17]:
ret["response_message"]

Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"where_clause":"WHERE city = \'New York\' AND since_date <= \'2023-01-01\'"}', name='query_names'), id='call_AR1LI5hcO09BrvYxyS81zvOT', type='function')], function_call=None)